In [2]:
import pandas as pd

# Load the cleaned dataset from Notebook 01
df = pd.read_csv("../data/reviews_cleaned.csv")

df.head()


,reviewText,overall,sentiment
0,If you are a serious violin student on a budge...,5.0,1
1,This is and excellent edition and perfectly tr...,5.0,1
2,Perfect for someone who is an opera fan or a w...,5.0,1
3,How many Nocturnes does it contain? All of the...,1.0,0
4,"this was written for Carin Levine in 2008, but...",5.0,1


In [3]:
X = df['reviewText']
y = df['sentiment']


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

len(X_train), len(X_test)


(367814, 91954)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),
    stop_words='english'
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.shape, X_test_tfidf.shape


((367814, 5000), (91954, 5000))

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import os

os.makedirs("../artifacts", exist_ok=True)

df = pd.read_csv("../data/reviews_cleaned.csv")

# CREATE LABEL (this was missing before)
df["overall_sentiment"] = df["overall"].apply(
    lambda x: "positive" if x >= 4 else "negative"
)

X = df["reviewText"]
y = df["overall_sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

tfidf = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1,2),
    stop_words="english"
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# SAVE EVERYTHING
joblib.dump(tfidf, "../artifacts/tfidf.joblib")
np.savez("../artifacts/X_train.npz", X_train_tfidf)
np.savez("../artifacts/X_test.npz", X_test_tfidf)
np.save("../artifacts/y_train.npy", y_train)
np.save("../artifacts/y_test.npy", y_test)
